In [27]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from cnn.minivggnet import MiniVGGNet
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 10
INIT_LR = 1e-2
BS = 32

# FASHION DATASET

In [28]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
print("..done")

trainX = trainX.reshape((trainX.shape[0], 28, 28))
testX = testX.reshape((testX.shape[0], 28, 28))

[INFO] loading Fashion MNIST...
..done


In [29]:
#compress and normalize the image by 2X2 plaquettes
def compress(image):
    lc = np.zeros((int(len(image)/2),len(image)))
    for i in range(int(len(image)/2)):
        lc[i,:] = image[2*i,:]+image[2*i+1,:]
    compress =  np.zeros((int(len(image)/2),int(len(image)/2)))   
    for i in range(int(len(image)/2)):
        compress[:,i] = lc[:,2*i]+lc[:,2*i+1]
    return compress/(np.max(image)*4)

In [32]:
comp_xtrain = np.zeros((len(trainX),int(trainX.shape[1]/2),int(trainX.shape[2]/2)))
for i, image in enumerate(trainX):
    comp_xtrain[i] = compress(image)
print("TRAIN: From ",trainX.shape)
print("to compressed ",comp_xtrain.shape)

comp_xtest = np.zeros((len(testX),int(testX.shape[1]/2),int(testX.shape[2]/2)))
for i, image in enumerate(testX):
    comp_xtest[i] = compress(image)    
print("TEST: From ",testX.shape)
print("to compressed ",comp_xtest.shape)


TRAIN: From  (60000, 28, 28)
to compressed  (60000, 14, 14)
TEST: From  (10000, 28, 28)
to compressed  (10000, 14, 14)


In [33]:
# reshape for CNN input (need to specify the depth ~ colour channels)
comp_xtrain = comp_xtrain.reshape((trainX.shape[0], 14, 14,1))
comp_xtest = comp_xtest.reshape((testX.shape[0], 14, 14,1))
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

# CNN training

In [34]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)

#model is a CNN, input parameters are the data shape
model = MiniVGGNet.build(width= 14, height= 14, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training model...")
H = model.fit(comp_xtrain, trainY,
	validation_data=(comp_xtest, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 185s 3ms/step - loss: 0.7974 - accuracy: 0.7213 - val_loss: 0.5015 - val_accuracy: 0.8196
Epoch 2/10
60000/60000 [==============================] - 178s 3ms/step - loss: 0.5380 - accuracy: 0.8006 - val_loss: 0.4414 - val_accuracy: 0.8355
Epoch 3/10
60000/60000 [==============================] - 179s 3ms/step - loss: 0.4957 - accuracy: 0.8164 - val_loss: 0.4232 - val_accuracy: 0.8428
Epoch 4/10
60000/60000 [==============================] - 176s 3ms/step - loss: 0.4703 - accuracy: 0.8248 - val_loss: 0.4050 - val_accuracy: 0.8493
Epoch 5/10
60000/60000 [==============================] - 180s 3ms/step - loss: 0.4482 - accuracy: 0.8332 - val_loss: 0.3982 - val_accuracy: 0.8496
Epoch 6/10
60000/60000 [==============================] - 181s 3ms/step - loss: 0.4395 - accuracy: 0.8360 - val_loss: 0.3943 - val_accuracy: 0.8510
Epoch 7/10


In [36]:
# make predictions on the test set
preds = model.predict(comp_xtest)
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
#plt.savefig("plot.png")

[INFO] evaluating network...
              precision    recall  f1-score   support

         top       0.82      0.84      0.83      1000
     trouser       0.95      0.96      0.96      1000
    pullover       0.76      0.82      0.79      1000
       dress       0.87      0.86      0.86      1000
        coat       0.75      0.76      0.75      1000
      sandal       0.98      0.93      0.96      1000
       shirt       0.68      0.59      0.63      1000
     sneaker       0.88      0.98      0.93      1000
         bag       0.97      0.97      0.97      1000
  ankle boot       0.97      0.92      0.94      1000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



# CIFAR-10 DATASET

In [38]:
from keras.datasets import cifar10

# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# X_train is 50000 rows of 3x32x32 values 

X_train = X_train.reshape(50000,32,32,3)
X_test = X_test.reshape(10000,32,32,3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.flatten()
y_test = y_test.flatten()

In [42]:
# normalize and compress the datasets
comp_xtrain = np.zeros((len(X_train),int(X_train.shape[1]/2),int(X_train.shape[2]/2),3))
for i,image in enumerate(X_train):
    for color in range(3):
        comp_xtrain[i,:,:,color] = compress(image[:,:,color])

comp_xtest = np.zeros((len(X_test),int(X_test.shape[1]/2),int(X_test.shape[2]/2),3))
for i,image in enumerate(X_test):
    for color in range(3):
        comp_xtest[i,:,:,color] = compress(image[:,:,color])

print(X_train.shape, 'train samples')
print(comp_xtrain.shape, 'compressed')
print(X_test.shape, 'test samples')
print(comp_xtest.shape, 'compressed')

(50000, 32, 32, 3) train samples
(50000, 16, 16, 3) compressed
(10000, 32, 32, 3) test samples
(10000, 16, 16, 3) compressed


In [43]:
# scale data to the range of [0, 1]
trainX = comp_xtrain
testX = comp_xtest
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(y_train, 10)
testY = np_utils.to_categorical(y_test, 10)
# initialize the label names
labelNames = ["airplane","automobile","bird","cat","deer",
              "dog","frog","horse","ship","truck"]

In [ ]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=16, height=16, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training model...")
H = model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 168s 3ms/step - loss: 1.9364 - accuracy: 0.3707 - val_loss: 1.4010 - val_accuracy: 0.4952
Epoch 2/10
50000/50000 [==============================] - 163s 3ms/step - loss: 1.4441 - accuracy: 0.4857 - val_loss: 1.1918 - val_accuracy: 0.5713
Epoch 3/10
50000/50000 [==============================] - 129s 3ms/step - loss: 1.2963 - accuracy: 0.5346 - val_loss: 1.0787 - val_accuracy: 0.6165
Epoch 4/10
50000/50000 [==============================] - 131s 3ms/step - loss: 1.2125 - accuracy: 0.5663 - val_loss: 1.0339 - val_accuracy: 0.6315
Epoch 5/10
50000/50000 [==============================] - 132s 3ms/step - loss: 1.1570 - accuracy: 0.5882 - val_loss: 0.9955 - val_accuracy: 0.6441
Epoch 6/10
50000/50000 [==============================] - 129s 3ms/step - loss: 1.1159 - accuracy: 0.6017 - val_loss: 0.9726 - val_accuracy: 0.6573
Epoch 7/10


In [ ]:
# make predictions on the test set
preds = model.predict(testX)
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig("plot.png")